In [25]:
%pylab inline
import numpy as np
from scipy import special
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


/home/roger/anaconda3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['gamma']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [26]:
METHOD_NAME = "nn_all_momentum"
DATA_FOLDER = "data/"
MODEL_FOLDER = "models/" + METHOD_NAME + "/"
OUTPUT_FOLDER = "outputs/"
TRAIN_FILE_PATH = DATA_FOLDER + "spam_train.csv"
TEST_FILE_PATH = DATA_FOLDER + "spam_test.csv"

In [27]:
if not os.path.exists(MODEL_FOLDER):
    os.makedirs(MODEL_FOLDER)
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

## Read Training Data

In [28]:
train_data = pd.read_csv(TRAIN_FILE_PATH, header=None)
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,1,0.0,0.00,1.13,0.0,0.37,0.00,0.00,0.0,0.00,...,0.0,0.145,0.000,0.436,0.000,0.0,1.792,55,147,0
1,2,0.0,0.00,0.60,0.0,0.00,0.60,0.00,0.0,0.60,...,0.0,0.143,0.047,0.191,0.143,0.0,2.041,31,196,1
2,3,0.0,0.00,0.48,0.0,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.000,0.450,0.000,0.0,1.138,4,41,0
3,4,0.0,0.51,0.00,0.0,0.51,0.51,1.02,0.0,0.00,...,0.0,0.142,0.000,0.071,1.212,0.0,7.025,130,281,1
4,5,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.64,...,0.0,0.116,0.000,0.232,0.000,0.0,1.551,6,45,0


In [29]:
x_data_rows = train_data[list(range(1, 58))].as_matrix()
y_data_rows = train_data[[58]].as_matrix()

## Select Data

## Preprocess Data

## Transform Data

In [30]:
# Normalize
x_train_means = np.mean(x_data_rows, axis=0)
x_train_stds =  np.std(x_data_rows, axis=0)
x_data_rows = (x_data_rows - x_train_means) / x_train_stds

In [31]:
# print(np.mean(x_data_rows, axis=0), np.std(x_data_rows, axis=0))

In [32]:
# add 1 in the last column, so that b in including in w
x_data_rows = np.c_[x_data_rows, np.ones(x_data_rows.shape[0]) ]

In [33]:
# pre-transpose x for matrix multiplication
x_data_rows_trans = np.transpose(x_data_rows)

In [34]:
x_row_num, x_col_num = x_data_rows.shape  # 4001, 58

## Train

In [35]:
# reset parameters
w = np.random.randn(x_col_num, 1)  # shape = (x_col_num, 1). not (1, x_col_num) for the convenience of matrix multiplication
total_epoch = 0

In [36]:
layer_1_node_num = 50

In [37]:
w_1 = np.random.randn(x_col_num, layer_1_node_num)  # dot(x, w_1) output a x_row_num * layer_1_node_num matrix
w_2 = np.random.randn(layer_1_node_num, 1)  # dot(layer_1_output, w_2) output a x_row_num * 1 matrix

In [38]:
# momentum
gamma = 0.9
vw = np.zeros((x_col_num, 1))

In [39]:
lr = (10) / x_row_num 

In [40]:
def sigmoid_deriv(x):  # input may be an matrix
    return x * (1-x)  # element-wise

In [41]:
# training iterations

# special.expit is sigmoid

while True:
    layer_0_output = x_data_rows
    layer_1_output = special.expit(np.dot(layer_0_output, w_1))  # shape = (x_row_num, layer_1_node_num)
    layer_2_output = special.expit(np.dot(layer_1_output, w_2))  # shape = (x_row_num, 1)
    y_diff_rows = (y_data_rows - layer_2_output)  # shape = (x_row_num, 1)
    
    # backpropagation
    layer_2_error = y_diff_rows  # shape = (x_row_num, 1)
    layer_2_delta = layer_2_error * sigmoid_deriv(layer_2_output)  # shape = (x_row_num, 1)
    
    layer_1_error = layer_2_delta.dot(w_2.T)  # shape = (x_row_num, layer_1_node_num)
    layer_1_delta = layer_1_error * sigmoid_deriv(layer_1_output)  # shape = (x_row_num, layer_1_node_num)
    
    w_2 += lr * (layer_1_output.T).dot(layer_2_delta)  # shape = (layer_1_node_num, 1)
    w_1 += lr * (layer_0_output.T).dot(layer_1_delta)  # shape = (x_row_num, layer_1_node_num)
    

    # write model
    if total_epoch % 5000 == 0:
        rms_error = np.mean(np.square(y_diff_rows))
        
        model_file_info = "epo" + str(total_epoch) + "_err" + str(rms_error)[:8]
        print(model_file_info)
        print('rms_error', rms_error)
        
#         np.savez(MODEL_FOLDER + model_file_info, w=w, vw=vw, lr=lr, total_epoch=total_epoch, x_train_means=x_train_means, x_train_stds=x_train_stds)

#     vw = lr * w_grad + gamma * vw
#     w = w - vw

    total_epoch += 1
    

epo0_err0.591059
rms_error 0.591059208245
epo5000_err0.011723
rms_error 0.0117232665283
epo10000_err0.009335
rms_error 0.00933546600327
epo15000_err0.008651
rms_error 0.00865179611418
epo20000_err0.008123
rms_error 0.00812387962619
epo25000_err0.007947
rms_error 0.00794795770983
epo30000_err0.007830
rms_error 0.007830398846
epo35000_err0.007744
rms_error 0.00774422191918
epo40000_err0.007675
rms_error 0.00767561435278
epo45000_err0.007617
rms_error 0.00761759268155
epo50000_err0.007567
rms_error 0.00756703944653
epo55000_err0.007522
rms_error 0.00752239660135
epo60000_err0.007482
rms_error 0.00748279137809
epo65000_err0.007447
rms_error 0.00744757875237
epo70000_err0.007416
rms_error 0.00741602258076
epo75000_err0.007387
rms_error 0.00738744502474
epo80000_err0.007361
rms_error 0.00736122628878
epo85000_err0.007336
rms_error 0.00733682228705
epo90000_err0.007313
rms_error 0.0073138395796
epo95000_err0.007291
rms_error 0.00729193773989
epo100000_err0.007270
rms_error 0.0072706160057
epo

KeyboardInterrupt: 

## Test

In [49]:
test_data = pd.read_csv(TEST_FILE_PATH, header=None)
test_data.shape

(600, 58)

In [50]:
_id = test_data[0].apply(str)

In [51]:
x_test_data_rows = test_data[list(range(1, 58))].as_matrix()

In [52]:
# read model

In [53]:
# normalize
x_test_data_rows = (x_test_data_rows - x_train_means) / x_train_stds
# add 1
x_test_data_rows = np.c_[x_test_data_rows, np.ones(x_test_data_rows.shape[0]) ]

In [54]:
def to_bool(x):
    return 1 if x > 0.5 else 0

to_bool = np.vectorize(to_bool)

In [55]:
# test
test_layer_1_output = special.expit(np.dot(x_test_data_rows, w_1))
test_layer_2_output = special.expit(np.dot(test_layer_1_output, w_2))
y_test_rows = to_bool(test_layer_2_output)

y_test_series = pd.Series(y_test_rows.flatten())

# concat id and y
output = pd.concat([_id, y_test_series], axis=1)
output.columns=["id","label"]

# write file
output.to_csv(OUTPUT_FOLDER + "output_" + METHOD_NAME + "_" + model_file_info + ".csv", index=False)